In [15]:
pip install sastrawi

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd 
import string 
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

In [17]:
dataset_sg = pd.read_csv('data.csv')

In [18]:
#Case Folding
data_baru = dataset_sg['Ulasan'].str.lower()
dataset_sg['Ulasan'] = data_baru

In [19]:
#Menghapus tag <br>
dataset_sg['Ulasan'] = dataset_sg['Ulasan'].replace("<br>", " ")

In [20]:
#Hapus angka dan simbol
def hapus_angka_simbol(data):
    data_bersih = re.sub('[^a-zA-Z]', ' ', data)
    return data_bersih

dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(hapus_angka_simbol)

In [21]:
#baca slangword
slang_dict = pd.read_excel('sw.xlsm')

# Buat dictionary untuk mapping slangword dan standar
slang_map = dict(zip(slang_dict['slang'], slang_dict['formal']))

In [22]:
# Membuat fungsi untuk mencari dan mengganti slangword
def hapus_slangword(text):
    for key in slang_map:
        text = re.sub(rf"\b{key}\b", slang_map[key], text)
    return text

# Mengganti slangword di kolom tertentu
dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(hapus_slangword)

In [23]:
#Tokenizing with unigram
def token_unigram(data):
    datas = word_tokenize(data)
    return [token for token in datas if len(token) > 1]

dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(token_unigram)

In [24]:
#Remove Indonesian Stopword
nltk.download('stopwords')
indonesia_stop = set(stopwords.words('indonesian'))
indonesia_stop.discard('tidak')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rivaldo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def hilang_stopword(data):
    delete_stop = [token for token in data if token not in indonesia_stop]
    return_data = ' '.join(delete_stop)
    return return_data
dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(hilang_stopword)

In [26]:
#Stemming with Sastrawi
f = StemmerFactory()
stem = f.create_stemmer()

def stemming(data):
    data = stem.stem(data)
    return data

dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(stemming)

In [27]:
dataset_sg['Ulasan']

0                              sayang jalan masuk sempit
1             wisata bagus buang sampah sembarang sayang
2                    loket parkir biar tidak macet cepat
3      orang kota sibuk sawah pandang tarik duduk san...
4      wow alam pandang asri dingin kayak pemandangan...
                             ...                        
669                               jalan utama baik mobil
670    panas banget baik pandang nya bagus air segar ...
671                              tutup pagi pakai banget
672    bagus alami sayang nya batu alas kolam renang ...
673            bagus pandang sayang makan tidak variatif
Name: Ulasan, Length: 674, dtype: object

In [28]:
dataset_sg.to_csv('data_ap_n_a.csv', index=False)